- 수집 대상 사이트 주소
- https://pjt3591oo.github.io

In [1]:
import numpy as np
import pandas as pd
import bs4, requests, time, os
from IPython.display import clear_output

In [2]:
# 요청 함수.
def getSource(site) :
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청.
    response = requests.get(site, headers = header_info)
    # print(response.text)
    
    # bs4 객체 생성.
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수.
def getData(soup) :
    # print(soup)
    
    # 데이터를 담을 딕셔너리 생성.
    data_dict = {
        '큰제목'   : [],
        '작은제목' : [],
        '날짜'     : [],
        '작성자'   : []
                }
    
    # 데이터가 있는 영역 전체를 가져오기.
    a1 = soup.select_one('body > main > div > div')
    # print(a1)
    
    # 데이터가 있는 p 태그를 가져오기.
    # a2 = a1.select('div.p')
    a2 = soup.select('body > main > div > div > div')[:-1]
    # print(a2)
    
    
    # div 태그의 수 만큼 반복.
    for div in a2 :
        # 큰 제목을 가져오기.
        a3 = div.select_one('h3 > a')
        data1 = a3.text.strip()
        
        # 작은 제목을 가져오기.
        a4 = div.select_one('h4')
        data2 = a4.text.strip()
        
        # 만약 길이가 0인 문자열이 추출됬다면 결측치로 변경.
        if len(data2) == 0 :
            data2 = np.nan
        # print(data2)
        
        # 날짜, 이름을 가져오기.
        a5 = div.select_one('p > span')
        data3 = a5.text.strip()
        # print(data3)
        data4 = data3.split('|')
        data5 = data4[0].strip()
        data6 = data4[1].strip()
        
        # print(data1)
        # print(data2)
        # print(data5)
        # print(data6)
        
        # 딕셔너리에 데이터를 담기.
        data_dict['큰제목'].append(data1)
        data_dict['작은제목'].append(data2)
        data_dict['날짜'].append(data5)
        data_dict['작성자'].append(data6)
        
    # 데이터 프레임 생성.
    df1 = pd.DataFrame(data_dict)
    # display(df1)

    if os.path.exists('data.csv') == False :
        # 파일이 없을 경우.
        df1.to_csv('data.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('data.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

In [4]:
# 다음페이지 존재 여부 확인하는 함수.
def getNextPage(soup) :
    # next 버튼 태그를 가져옴.
    a1 = soup.select_one('body > main > div > div > div.pagination > ul > li:nth-child(6) > a')
    # print(a1)
    
    # 만약 가져온 것이 존재한다면
    if a1 != None :
        # a 태그의 href 속성 값을 가져와서 반환.
        href = a1.attrs['href']
        # print(href)
        return href
    
    else :
        return None

In [5]:
site = 'https://pjt3591oo.github.io'
page = ''

while True :
    # 3초 대기.
    time.sleep(3)
    
    # 기존 출력된 것 청소.
    clear_output(wait=True)
    
    # 페이지 요청.
    soup = getSource(site + page)
    
    # 데이터 수집.
    getData(soup)
    
    # 다음 페이지  존재 확인.
    page = getNextPage(soup)
    
    # 다음 페이지가 없다면 중단.
    if page == None :
        print('수집완료')
        break

수집완료


In [6]:
# 데이터 프레임 생성.
df = pd.read_csv('data.csv')
df.head()

,큰제목,작은제목,날짜,작성자
0,[programming] [react] react 작업환경 설,react를 시작하기 전에 환경셋팅을 해보자,2017-05-20 06:29:05 +0000,박정태
1,[programming] drag and drop을 이용하여 파일 업로드를 해보자,"query의 외부 라이브러리가 아닌 drag, drop 이벤트를 활용하여 기능 구현해보기",2017-05-09 23:47:05 +0000,박정태
2,"[database] mysqldump를 이용하여 데이터 백업, 복원하기",mysqldump를 이용하여 디비 백업과 source를 이용하여 데이터 복원을 해보자,2017-05-04 05:33:05 +0000,박정태
3,[database] mysql 원격접속하는 방법,"mysql 디비설정, 유저설정을 통해 원격접속",2017-05-03 11:43:05 +0000,박정태
4,"[node.js] 파일 리더기 만들기 - 사용 모듈 정리, pdf와 hwp 구조","docx, hwp, pdf 파일",2017-05-01 06:02:05 +0000,박정태
